In [154]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from scipy.optimize import minimize

from sklearn import datasets
from sklearn.preprocessing import PolynomialFeatures

In [186]:
data = datasets.load_iris()
data.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename'])

In [187]:
def FeatureNormalize(X):
    mu = X.mean(axis=0)
    sigma = X.std(axis=0)
    [m,n]=X.shape
    t = np.ones((m,n))
    
    Xnorm = (X-(t*mu))/(t*sigma)
    
    return(Xnorm)

# logisticRegression with features:
    1. Linear
    2. Polynomial

In [188]:
# # Feature Linear
# X = np.c_[np.ones((data['data'].shape[0])),data['data']]
# y = data['target'].reshape(-1,1)

# [m,n] = X.shape
# initialTheta = np.zeros((n,1))

In [189]:
# Feature Polynomial
X = FeatureNormalize(data['data'])
# X = data['data']
poly = PolynomialFeatures(6)
X = poly.fit_transform(X)

y = data['target'].reshape(-1,1)

[m,n] = X.shape
initialTheta = np.zeros((n,1))

In [190]:
X.shape

(150, 210)

In [191]:
y.shape

(150, 1)

In [192]:
initialTheta.shape

(210, 1)

# training data

In [193]:
def sigmoid(z):
    return(1/(1+np.exp(-z)))

In [194]:
def lrCostFunction(theta,reg,X,y):
    [m,n] = X.shape
    h = sigmoid(X.dot(theta))
    
    cfReg = (reg/2*m)*np.sum(np.square(theta[1:]))
    J = (-1/m) + (np.log(h).T.dot(y) + np.log(1-h).T.dot(1-y)) + cfReg    
    
    return J

In [195]:
def lrgradient(theta,reg,X,y,alpha,num_iter):
    [m,n] = X.shape
    for i in range(num_iter):
        h = sigmoid(X.dot(theta.reshape(-1,1)))
        gradReg = (reg/m)*np.r_[[[0]], theta[1:].reshape(-1,1)]
        grad = (1/m)*X.T.dot(h-y) + gradReg
        theta = theta - (alpha*grad)

    return(theta.flatten())

## oneVsAll

In [196]:
def getOptTheta(X,y,n_labels,reg,alpha,num_iter):
    [m,n] = X.shape
    initialTheta = np.zeros((n,1))
    allTheta = np.zeros((n_labels,n))
    test=[]
    for c in np.arange(n_labels):
        t = lrgradient(initialTheta,reg,X,(y==c)*1,alpha,num_iter)
        allTheta[c] = t
        test.append(t)
    return(allTheta)

In [197]:
#def getOptTheta(X,y,n_labels,reg,alpha,num_iter)
thetaOpt = getOptTheta(X, y, 3, 10, 0.1, 30000)
thetaOpt
#numlabel, features

array([[-4.82735927e+00, -4.87782181e-02,  4.64269879e-02,
        -7.50132523e-02, -6.40071534e-02,  1.55576960e-02,
        -2.28808031e-02,  3.25642825e-02,  2.53338254e-02,
        -1.16504898e-03, -2.53609430e-02, -2.13019469e-02,
         4.59011687e-02,  3.50895267e-02,  2.40597488e-02,
        -4.35035736e-02,  2.51329443e-02, -5.19781295e-02,
        -4.35059148e-02, -1.61738455e-02,  2.90980968e-02,
         2.21961001e-02, -6.42765536e-02, -5.36676464e-02,
        -4.67052863e-02,  3.53110688e-02, -3.28021610e-02,
        -2.68654072e-02,  4.46551761e-02,  3.55430904e-02,
         2.99026916e-02, -9.05649643e-02, -7.67798836e-02,
        -6.85263599e-02, -6.48382422e-02,  6.38487991e-03,
        -1.42616099e-02,  2.12488796e-02,  1.65179530e-02,
         6.38330945e-03, -1.78148466e-02, -1.37070963e-02,
         3.40764461e-02,  2.61992532e-02,  1.92128905e-02,
        -1.37961368e-02,  1.56801670e-02,  1.24512504e-02,
        -2.74829279e-02, -2.20841492e-02, -1.82858409e-0

# prediction

In [183]:
def predictOneVsAll(all_theta, features):
    probs = sigmoid(X.dot(all_theta.T))
        
    # Adding one because Python uses zero based indexing for the 10 columns (0-9),
    # while the 10 classes are numbered from 1 to 10.
    return(np.argmax(probs, axis=1))

In [184]:
pred = predictOneVsAll(thetaOpt, X)
print('Regularization : 10')
print('Training set accuracy: {} %'.format(np.mean(pred == y.ravel())*100))

Regularization : 10
Training set accuracy: 90.66666666666666 %


In [180]:
pred = predictOneVsAll(thetaOpt, X)
print('Regularization : 0/ tanpa regularisasi')
print('Training set accuracy: {} %'.format(np.mean(pred == y.ravel())*100))

Training set accuracy: 98.0 %


In [198]:
pred = predictOneVsAll(thetaOpt, X)
print('degree Polynomial: 6')
print('Training set accuracy: {} %'.format(np.mean(pred == y.ravel())*100))

degree Polynomial: 6
Training set accuracy: 96.66666666666667 %


In [418]:
# OPTIMUM SET: 
# reg = 10
# normalisasi
# degree poly = 6